# Project Title:
**AI Guardian: An AI-Powered Consumer Protection Assistant**

## Project Overview:
The AI Guardian project aims to develop an AI-powered assistant designed to help consumers navigate the digital marketplace safely and confidently. The assistant will focus on providing reliable information, detecting scams and misinformation, and offering support for consumer rights and redress mechanisms.

## Key Features:
1. **Trustworthy Information Source**:
   - **Verified Answers**: Use natural language processing (NLP) to provide consumers with verified and citation-backed answers to their queries.
   - **Bias Detection**: Implement algorithms to detect and mitigate regional or cultural biases in information provided.

2. **Scam and Misinformation Detection**:
   - **Real-time Scam Alerts**: Use machine learning to identify and alert users about potential scams in real-time, whether in emails, advertisements, or online shopping platforms.
   - **Misinformation Filtering**: Analyze content from various sources and flag misinformation, providing users with accurate and verified alternatives.

3. **Consumer Rights Education**:
   - **Interactive Learning Modules**: Offer interactive and engaging modules to educate consumers about their rights, especially in relation to AI and digital services.
   - **Guided Redress Mechanisms**: Provide step-by-step guidance on how consumers can seek redress for issues with AI technologies or digital services.

4. **Personalized Recommendations and Alerts**:
   - **Custom Alerts**: Based on user preferences and past behavior, the assistant can offer personalized alerts about new scams or critical updates in consumer protection laws.
   - **Safe Shopping Recommendations**: Recommend safe and reliable online marketplaces and vendors based on AI analysis of consumer reviews and ratings.

## Import Libraries

In [1]:
import os 
import dotenv
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory
from typing import Dict
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

## Load API KEYS

In [2]:
dotenv.load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Initializing Chat Model
Let's initialize the chat model which will serve as the chatbot's brain:

In [3]:
Model = "gpt-3.5-turbo-1106"
# Model = "gpt-4o"

chat = ChatOpenAI(model=Model, temperature=0.2)

## Invoking The Model
If we invoke our chat model, the output is an AIMessage:

In [4]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency LLMs."})

AIMessage(content='Low latency LLMs, or Low-Latency Machine Learning Models, are important for a variety of real-time applications where quick decision-making is crucial. These models are designed to minimize the time it takes to process and respond to incoming data, making them well-suited for use cases such as autonomous vehicles, real-time fraud detection, high-frequency trading, and interactive user interfaces.\n\nBy reducing the latency, or the delay between input and output, low latency LLMs can provide faster and more accurate responses, leading to improved user experiences and better performance in time-sensitive applications. This can be especially critical in scenarios where split-second decisions can have significant impact, such as in healthcare, emergency response, and financial trading.\n\nOverall, low latency LLMs play a key role in enabling real-time decision-making and enhancing the efficiency and effectiveness of various applications across different industries.', res

In [5]:
chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)

AIMessage(content="J'adore la programmation.", response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 19, 'total_tokens': 27}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e771c40e-d476-420a-a680-8205146c3383-0')

## Model State
The model on its own does not have any concept of state. For example, if you ask a followup question:

In [6]:
chat.invoke([HumanMessage(content="What did you just say?")])

AIMessage(content='I said, "What did you just say?"', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9323776c-b84b-4482-ac2b-b4199e6221f6-0')

We can see that it doesn't take the previous conversation turn into context, and cannot answer the question.

To get around this, we need to pass the entire conversation history into the model. Let's see what happens when we do that:

In [7]:
chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        ),
        AIMessage(content="J'adore la programmation."),
        HumanMessage(content="What did you just say?"),
    ]
)

AIMessage(content='I said "J\'adore la programmation" which means "I love programming" in French.', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 41, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dba0c014-e397-457b-adfc-9af166fc2ad9-0')

And now we can see that we get a good response!

This is the basic idea underpinning a chatbot's ability to interact conversationally.

## Prompt templates
Let's define a prompt template to make formatting a bit easier. We can create a chain by piping it into the model:

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

The MessagesPlaceholder above inserts chat messages passed into the chain's input as chat_history directly into the prompt. Then, we can invoke the chain like this:

In [9]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

AIMessage(content='I said "J\'adore la programmation," which means "I love programming" in French.', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 61, 'total_tokens': 82}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fa897a12-0b23-4b36-b8cb-2efc8e2d00fa-0')

## Message history
As a shortcut for managing the chat history, we can use a MessageHistory class, which is responsible for saving and loading chat messages. There are many built-in message history integrations that persist messages to a variety of databases, but for this quickstart we'll use a in-memory, demo message history called ChatMessageHistory.

In [10]:
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("hi!")

demo_ephemeral_chat_history.add_ai_message("whats up?")

demo_ephemeral_chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

Once we do that, we can pass the stored messages directly into our chain as a parameter:

In [11]:
demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_ephemeral_chat_history.messages})

response

AIMessage(content='The translation of "I love programming" in French is "J\'adore la programmation."', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 53, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8ed46707-6e9d-4496-9717-994916ce57d9-0')

In [12]:
demo_ephemeral_chat_history.add_ai_message(response)

demo_ephemeral_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_ephemeral_chat_history.messages})

AIMessage(content='I said, "J\'adore la programmation," which means "I love programming" in French.', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 87, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cbddebc1-5e55-4d1b-960f-89149ea9a454-0')

**And now we have a basic chatbot!**

```While this chain can serve as a useful chatbot on its own with just the model's internal knowledge, it's often useful to introduce some form of retrieval-augmented generation, or RAG for short, over domain-specific knowledge to make our chatbot more focused. We'll cover this next.```

## Retrievers
We can set up and use a Retriever to pull domain-specific knowledge for our chatbot. To show this, let's expand the simple chatbot we created above to be able to answer questions about LangSmith.

We'll use the **Consumer Day and related website** as source material and store it in a vectorstore for later retrieval. Note that this example will gloss over some of the specifics around parsing and storing a data source.

To go Deeper - ```https://python.langchain.com/v0.1/docs/use_cases/question_answering/```

In [18]:
# Import the necessary modules
from bs4 import BeautifulSoup
import requests
from langchain_text_splitters.base import Document

# Define the load_data function
def load_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to fetch data from {url}")

# Define a list of URLs
urls = [
    "https://unctad.org/topic/competition-and-consumer-protection/un-guidelines-for-consumer-protection",
    "https://www.eccnet.eu/news/artificial-intelligence-how-do-we-protect-consumer-rights-ai-applications",
    "https://www.eccnet.eu/sites/default/files/2023-01/Digital_fairness_fitness_check.pdf",
    # "https://www.investopedia.com/articles/pf/10/know-your-consumer-protection-laws.asp",
    "https://www.fcc.go.tz/pdf/GUIDELINES2CONSUMER%20PROTECTION%20PROVISIONS%20IN%20THE%20FCA_20170707.pdf",
    "https://ncdrc.nic.in/bare_acts/1_1_2.html#:~:text=Example%20%3A%20A%20was%20running%20a,the%20photocopier%20for%20commercial%20use.",
    "https://www.consumerprotection.govt.nz/general-help/consumer-laws/consumer-guarantees-act",
    "https://www.law.ac.uk/employability/legal-practice-areas/consumer-law/#:~:text=Consumer%20law%20provides%20protection%20to,and%20regulations%20of%20this%20directive.",
    # "https://labourguide.co.za/general/the-consumer-protection-act-your-guide-to-consumer-rights-a-how-to-protect-them#:~:text=Suppliers%20are%20obliged%20to%20replace,failed%20or%20are%20considered%20unsafe.",
    "https://www.citizensinformation.ie/en/consumer/consumer-laws/your-consumer-rights/#:~:text=You%20are%20protected%20against%20unfair,Use%20false%20or%20misleading%20descriptions",
    "https://en.wikipedia.org/wiki/Consumer_protection",
    "https://www.nyulawglobal.org/globalex/International_Law_Consumer_Protection.html#:~:text=The%20United%20Nations%20Guidelines%20for,of%20economic%20interests%20of%20consumers.",
    # "https://www.fcc.go.tz/#:~:text=Welcome%20to%20FCC&text=8%20of%202003%20(FCA)%20with,unfair%20and%20misleading%20market%20conduct.",
    "https://www.fcc.go.tz/documents/public-register/guidelines/GUIDELINES_fcc_merger_guidelines_tanzania_20170705.pdf",
    "https://www.fcc.go.tz/documents/public-register/guidelines/GUIDELINES_COMPETITION%20COMPLAINT%20INVESTIGATION_20170703.pdf",
    "https://www.fcc.go.tz/documents/public-register/guidelines/MUONGOZO%20KUHUSU%20UCHUNGUZI%20WA%20MAKOSA%20YANAYOKIUKA%20USHINDANI%20-TOLEO%20LA%202004_20170703.pdf",
    "https://www.fcc.go.tz/documents/public-register/consumer-form/FCC_SFC%20No.5_Submission%20of%20Amendment%20OR%20Replacement%20of%20Standard%20Form%20Contracts.pdf",
    "https://www.fcc.go.tz/documents/public-register/consumer-form/FCC_SFC%20No.4_Reply%20to%20Notification%20on%20Complaint%20of%20the%20use%20of%20unfair%20Terms.pdf",
    "https://www.fcc.go.tz/documents/public-register/consumer-form/FCC_SFC%20No.3_Notification%20on%20Complaint%20Filed%20with%20the%20FCC.pdf",
    "https://www.fcc.go.tz/documents/public-register/consumer-form/FCC_SFC%20No.2_Complaint%20Form.pdf",
    "https://www.fcc.go.tz/documents/public-register/consumer-form/FCC_SFC%20No.1_Registration%20of%20Standard%20Form%20Contract.pdf",
    "https://www.fcc.go.tz/rules-regulations-orders",
    # Add more URLs here
]

# Iterate through the list of URLs and load data from each
documents = []
for url in urls:
    data = load_data(url)
    text_content = data.get_text()
    document = Document(page_content=text_content)
    documents.append(document)

# Now you can use the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(documents)

Next, we split it into smaller chunks that the LLM's context window can handle and store it in a vector database:

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)

Then we embed and store those chunks in a vector database:

### Creating and Saving and Loading the Embeddings

In [19]:
embedding = OpenAIEmbeddings()
persist_dir = 'Embeddings'

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_dir)

And finally, let's create a retriever from our initialized vectorstore:

In [20]:
# Load the vector store
vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embedding)

# Define the retriever
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("Summarise the Guidelines for Consumer Protection")

docs

[Document(page_content='Download:\n\nThe United Nations Guidelines for Consumer Protection\n\t| English | French | Spanish | Arabic | Chinese | Russian |\n\n\n\n\n\n\n\n\n\nRight navigation\n\n\nCompetition and Consumer Protection\n\n\nIntergovernmental Group of Experts on Competition Law and Policy\n\n\nIntergovernmental Group of Experts on Consumer Protection Law and Policy\n\n\nVoluntary Peer Review of Competition Law and Policy\n\n\nVoluntary Peer Review of Consumer Protection Law and Policy', metadata={'description': 'The United Nations Guidelines for Consumer Protection (UNGCP) are "a valuable set of principles for setting out the main characteristics of effective consumer protection legislation, enforcement institutions and redress systems and for assisting interested Member States in formulating and enforcing domestic and regional laws, rules and regulations that are suitable to their own economic and social and environmental circumstances, as well as promoting international en

We can see that invoking the retriever above results in some parts of the UN guiedins for Consumer Protection document that contain information about testing that our chatbot can use as context when answering questions.

## Handling documents
Let's modify our previous prompt to accept documents as context. We'll use a create_stuff_documents_chain helper function to "stuff" all of the input documents into the prompt, which also conveniently handles formatting. We use the ChatPromptTemplate.from_messages method to format the message input we want to pass to the model, including a MessagesPlaceholder where chat history messages will be directly injected:

In [21]:
# chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

We can invoke this document_chain with the raw documents we retrieved above:

In [22]:
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("Summarise the Guidelines for Consumer Protection")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

'The United Nations Guidelines for Consumer Protection provide a framework for consumer protection policies, including fair business practices, product safety, and consumer education. They aim to promote and protect the economic, physical, and social well-being of consumers. The guidelines cover areas such as protection of vulnerable and disadvantaged consumers, product safety, and consumer education and information. They also emphasize the importance of effective enforcement mechanisms and international cooperation in consumer protection. The guidelines are available in multiple languages for wider accessibility.'

Awesome! We see an answer synthesized from information in the input documents.

## Creating a retrieval chain
Next, let's integrate our retriever into the chain. Our retriever should retrieve information relevant to the last message we pass in from the user, so we extract it and use that as input to fetch relevant docs, which we add to the current chain as context. We pass context plus the previous messages into our document chain to generate a final answer.

We also use the RunnablePassthrough.assign() method to pass intermediate steps through at each invocation. Here's what it looks like:

In [23]:
def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [24]:
response = retrieval_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)

response

{'messages': [HumanMessage(content='Summarise the Guidelines for Consumer Protection')],
 'context': [Document(page_content='Download:\n\nThe United Nations Guidelines for Consumer Protection\n\t| English | French | Spanish | Arabic | Chinese | Russian |\n\n\n\n\n\n\n\n\n\nRight navigation\n\n\nCompetition and Consumer Protection\n\n\nIntergovernmental Group of Experts on Competition Law and Policy\n\n\nIntergovernmental Group of Experts on Consumer Protection Law and Policy\n\n\nVoluntary Peer Review of Competition Law and Policy\n\n\nVoluntary Peer Review of Consumer Protection Law and Policy', metadata={'description': 'The United Nations Guidelines for Consumer Protection (UNGCP) are "a valuable set of principles for setting out the main characteristics of effective consumer protection legislation, enforcement institutions and redress systems and for assisting interested Member States in formulating and enforcing domestic and regional laws, rules and regulations that are suitable to

Nice! Our chatbot can now answer domain-specific questions in a conversational way.

As an aside, if you don't want to return all the intermediate steps, you can define your retrieval chain like this using a pipe directly into the document chain instead of the final .assign() call:

In [25]:
retrieval_chain_with_only_answer = (
    RunnablePassthrough.assign(
        context=parse_retriever_input | retriever,
    )
    | document_chain
)

retrieval_chain_with_only_answer.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    },
)

'The United Nations Guidelines for Consumer Protection provide a framework for consumer protection policies, including fair business practices, product safety, and consumer education. The guidelines aim to promote and protect the economic, physical, and social well-being of consumers. They cover areas such as protection of vulnerable and disadvantaged consumers, sustainable consumption, and consumer dispute resolution. The guidelines are available in multiple languages, including English, French, Spanish, Arabic, Chinese, and Russian.'

## Query transformation
There's one more optimization we'll cover here - in the above example, when we asked a followup question, tell me more about that!, you might notice that the retrieved docs don't directly include information about testing. This is because we're passing tell me more about that! verbatim as a query to the retriever. The output in the retrieval chain is still okay because the document chain retrieval chain can generate an answer based on the chat history, but we could be retrieving more rich and informative documents:

In [26]:
retriever.invoke("Summarise the Guidelines for Consumer Protection")

[Document(page_content='Download:\n\nThe United Nations Guidelines for Consumer Protection\n\t| English | French | Spanish | Arabic | Chinese | Russian |\n\n\n\n\n\n\n\n\n\nRight navigation\n\n\nCompetition and Consumer Protection\n\n\nIntergovernmental Group of Experts on Competition Law and Policy\n\n\nIntergovernmental Group of Experts on Consumer Protection Law and Policy\n\n\nVoluntary Peer Review of Competition Law and Policy\n\n\nVoluntary Peer Review of Consumer Protection Law and Policy', metadata={'description': 'The United Nations Guidelines for Consumer Protection (UNGCP) are "a valuable set of principles for setting out the main characteristics of effective consumer protection legislation, enforcement institutions and redress systems and for assisting interested Member States in formulating and enforcing domestic and regional laws, rules and regulations that are suitable to their own economic and social and environmental circumstances, as well as promoting international en

In [27]:
retriever.invoke("tell me more about that!")

[Document(page_content='8�\x13!!�'),
 Document(page_content='8�\x13!!�'),
 Document(page_content='8�\x13!!�'),
 Document(page_content=',�.��')]

To get around this common problem, let's add a query transformation step that removes references from the input. We'll wrap our old retriever as follows:

In [28]:
# We need a prompt that we can pass into an LLM to generate a transformed search query

# chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

Now let's recreate our earlier chain with this new query_transforming_retriever_chain. Note that this new chain accepts a dict as input and parses a string to pass to the retriever, so we don't have to do additional parsing at the top level:

In [29]:
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

demo_ephemeral_chat_history = ChatMessageHistory()

And finally, let's invoke it!

In [30]:
demo_ephemeral_chat_history.add_user_message("Summarise the Guidelines for Consumer Protection")

response = conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages},
)

demo_ephemeral_chat_history.add_ai_message(response["answer"])

response 

{'messages': [HumanMessage(content='Summarise the Guidelines for Consumer Protection'),
  AIMessage(content='The United Nations Guidelines for Consumer Protection provide a framework for consumer protection policies, laws, and regulations. They cover areas such as product safety, fair business practices, consumer education, and redress mechanisms. The guidelines aim to promote and protect the economic, physical, and social well-being of consumers worldwide. They are available in multiple languages, including English, French, Spanish, Arabic, Chinese, and Russian.')],
 'context': [Document(page_content='Download:\n\nThe United Nations Guidelines for Consumer Protection\n\t| English | French | Spanish | Arabic | Chinese | Russian |\n\n\n\n\n\n\n\n\n\nRight navigation\n\n\nCompetition and Consumer Protection\n\n\nIntergovernmental Group of Experts on Competition Law and Policy\n\n\nIntergovernmental Group of Experts on Consumer Protection Law and Policy\n\n\nVoluntary Peer Review of Compe

In [31]:
demo_ephemeral_chat_history.add_user_message("tell me more about that!")

conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

{'messages': [HumanMessage(content='Summarise the Guidelines for Consumer Protection'),
  AIMessage(content='The United Nations Guidelines for Consumer Protection provide a framework for consumer protection policies, laws, and regulations. They cover areas such as product safety, fair business practices, consumer education, and redress mechanisms. The guidelines aim to promote and protect the economic, physical, and social well-being of consumers worldwide. They are available in multiple languages, including English, French, Spanish, Arabic, Chinese, and Russian.'),
  HumanMessage(content='tell me more about that!')],
 'context': [Document(page_content='The United Nations Guidelines for Consumer Protection (UNGCP) are "a valuable set of principles for setting out the main characteristics of effective consumer protection legislation, enforcement institutions and redress systems and for assisting interested Member States in formulating and enforcing domestic and regional laws, rules and 

In [32]:
# Add user message to the chat history
demo_ephemeral_chat_history.add_user_message("thanks for that, so how is the consumers protection guaranteed")

# Invoke the conversational retrieval chain
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})

# Extract and print only the answer
print(response['answer'])


Consumer protection is guaranteed through the implementation of laws and regulations based on the United Nations Guidelines on Consumer Protection. These guidelines provide a framework for countries to establish and enforce policies that ensure product safety, fair business practices, consumer education, and effective redress mechanisms. By adhering to these guidelines, governments can create an environment where consumers are empowered, informed, and protected from unfair or harmful practices. Additionally, consumer protection agencies and organizations play a crucial role in monitoring and enforcing these guidelines to safeguard consumer rights.


And we now have a chatbot capable of conversational retrieval!

## Streamlit App

## Next steps
Next is to know how to build a conversational chatbot that can integrate past messages and domain-specific knowledge into its generations. There are  other optimizations we can make around this:

- **Memory management:** This includes a guide on automatically updating chat history, as well as trimming, summarizing, or otherwise modifying long conversations to keep your bot focused.
- **Retrieval:** A deeper dive into using different types of retrieval with the chatbot
- **Tool usage:** How to allows the chatbot to use tools that interact with other APIs and systems.